In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn import cross_validation, datasets, linear_model, metrics
%pylab inline

# Data

### Generate / Import dataset

In [ ]:
data_size = 200

In [298]:
data, target, coef = datasets.make_regression(n_samples = data_size,
                                              n_features = 2, 
                                              n_informative = 1, 
                                              n_targets = 1, noise = 5.,
                                              coef = True, random_state = 2)

In [299]:
#data = pd.read_csv('Data/weights_heights.csv', index_col='Index')
data_frame = pd.read_csv('Data/advertising.csv')
data_frame
#data.plot(y='TV', kind='hist', title='TV distribution')
#data.plot(y='Radio', kind='hist', title='Radio distribution')
#data.plot(y='Newspaper', kind='hist', title='Newspaper distribution')
#data.plot(y='Sales', kind='hist', title='Sales distribution')
#plt.show()
#sns.pairplot(data)
#plt.show()

features = ['TV'] # , 'Radio','Newspaper']
labels = ['Sales']
data = np.array(data_frame[features].values, dtype=float)[:data_size]
target = np.array(data_frame[labels].values, dtype=float)[:data_size]

#means, stds = X.mean(axis=0), X.std(axis=0)
#X = (X - means) / stds

#xsize = 2
#x = np.linspace(min(data[feature]),max(data[feature]),xsize).reshape((xsize,1))
#pylab.scatter(list(map(lambda x:x[0], data)), target, color = 'r')
#pylab.scatter(list(map(lambda x:x[1], data)), target, color = 'b')
#pylab.scatter(list(map(lambda x:x[2], data)), target, color = 'g')

### Split dataset into train & test samples

In [300]:
train_data, test_data, train_labels, test_labels = \
cross_validation.train_test_split(data, target, test_size = 0.3)

In [301]:
print('train_data: \n',train_data[:5])
print('train_labels: \n',train_labels[:5])

train_data: 
 [[  50. ]
 [ 184.9]
 [ 112.9]
 [  23.8]
 [ 290.7]]
train_labels: 
 [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]


# Run models and output

In [316]:
def print_result(coef, true, predict, cut = 5):
    print('w:\n',coef,'\n')
    print('true vs. prediction:\n',hstack((true,predict))[:cut],'\n...')

In [317]:
def plot_for_one_feature(train_data, train_labels, w, title):
    x = np.linspace(train_data.min(), train_data.max(), 2).reshape((2,1))
    plt.figure(figsize = (8,5))
    plt.plot(train_data, train_labels, 'o', markersize = 3)
    #print(x)
    #print(linear_regression_prediction(x, w))
    plt.plot(x, linear_regression_prediction(x,w), '-', linewidth = 4)
    plt.xlabel('feature')
    plt.ylabel('label')
    plt.title(title)
    plt.show()

### sklearn.linear_model.LinearRegression

In [318]:
linear_regressor = linear_model.LinearRegression()
linear_regressor.fit(train_data, train_labels)
linear_regressor.predict(test_data)
w = np.concatenate((linear_regressor.intercept_,linear_regressor.coef_[0]))
print_result(w,test_labels,linear_regressor.predict(test_data))
#plot_for_one_feature(train_data, train_labels, w, 'train')
#plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ 7.25202349  0.0452737 ] 

true vs. prediction:
 [[ 16.9         14.64521915]
 [ 22.4         16.09850501]
 [ 21.4         20.51269103]
 [  7.3          7.78172581]
 [ 24.7         17.22582021]] 
...


### Analytical OLS method

In [319]:
w = linear_regression_fit(train_data, train_labels, minimize='analytical')
print_result(w,test_labels,linear_regression_prediction(test_data, w))
#plot_for_one_feature(train_data, train_labels, w, 'train')
#plot_for_one_feature(test_data, test_labels, w, 'test')

[[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
[[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:
 [[ 7.25202349]
 [ 0.0452737 ]] 

true vs. prediction:
 [[ 16.9         14.64521915]
 [ 22.4         16.09850501]
 [ 21.4         20.51269103]
 [  7.3          7.78172581]
 [ 24.7         17.22582021]] 
...


### Numerical method using gradient descent

In [341]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels, minimize='st_grad_desc', loss_f='mse')
print_and_plot(w,test_labels,linear_regression_prediction(test_data,w))

[  -12.2  -2522.96]
X:
 [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
w:
 [ 0.00087143  0.18021143]


ValueError: shapes (140,3) and (2,) not aligned: 3 (dim 1) != 2 (dim 0)

### Numerical method using srochastic gradient descent

In [342]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels, minimize='st_grad_desc')[0]
print_and_plot(w,test_labels,linear_regression_prediction(w, test_data))

[  -12.2  -2522.96]
X:
 [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
w:
 [ 0.00087143  0.18021143]


ValueError: shapes (140,3) and (2,) not aligned: 3 (dim 1) != 2 (dim 0)

# Methods implementation

### Loss & cost functions

In [311]:
def mean_squared_error(y, y_pred):
    """
    Returns a mean squared error between real and predicted labels
    """
    y = np.array(y)
    y_pred = np.array(y_pred)
    mse = np.sum((y - y_pred)**2) / y.size
    return mse

def mean_absolute_error(y, y_pred):
    """
    Returns a mean absolute error between real and predicted labels
    """
    y = np.array(y)
    y_pred = np.array(y_pred)
    mae = np.sum(abs(y - y_pred)) / y.size
    return mae

def cost_function(w, X, y, loss_f = 'mse'):
    """
    Returns a cost function of a linear model with coefficients w
    oh features X and labels y.
    """
    if (loss_f == 'mae'):
        return mean_absolute_error(y, linear_regression_prediction(w, X))
    return mean_squared_error(y, linear_regression_prediction(w, X))

### Linear regression model

In [325]:
def linear_regression_fit(X, y, minimize = 'analytical', loss_f = 'mse'):
    """
    Returns weights as a least-squares solution to 
    linear matrix equation Xw = y.
    Solves this equation by computing a vector w
    that minimizes ||y-Xw||**2.
    If X is square and of full rank,
    then w is the “exact” solution of the equation.
    """
    n = X.shape[0]
    X = np.hstack((np.ones((n,1)),X))
    #print(X[:5])
    #print(y[:5])
    if (minimize == 'grad_desc'):
        # the loss function is 'mse' automatically
        w = gradient_descent(lambda w: cost_function(w, X, y, loss_f), w0)[0]
    elif (minimize == 'st_grad_desc'):
        # the loss function is 'mse' automatically
        w = stochastic_gradient_descent(X, y)
    elif (minimize == 'scipy_minimize'):
        w = minimize(lambda w: cost_function(w, X, y, loss_f), w0).x
    elif (minimize == 'analytical'):
        # the loss function is 'mse' automatically
        # X_t = X.transpose()
        # w = np.linalg.inv(X_t.dot(X)).dot(X_t).dot(y)
        w = np.linalg.lstsq(X,y)[0]
    else:
        #raise
        print('error: incorrect minimization method')
        return
    return w
    

def linear_regression_prediction(X, w):
    """
    Returns a linear regression model prediction labels for objects in matrix 
    X using weights w:
    y_pred = (X,w)
    """
    n = X.shape[0]
    X = np.hstack((np.ones((n,1)),X))
    return np.dot(X,w)

### Gradient descent

In [313]:
def grad(f,x,h=1e-3):
    dim = x.size
    I = np.eye(dim)
    H = h * np.ones(dim)
    return np.array([(f(x + H * I_vec) - f(x - H * I_vec)) / 
                     (2.0 * (H @ I_vec)) for I_vec in I])

def stop(f,x_old,x_new,eps = 1e-5):
    stop1 = np.linalg.norm(x_new-x_old) <= eps
    stop1 = abs(f(x_new)-f(x_old)) <= eps
    stop2 = np.linalg.norm(grad(f,x_new), ord=x_new.size) <= eps
    return stop1 #and stop2 and stop3

def alpha_split(f,x,step,b=0.1,l=0.5):
    alpha = b
    q = 0.1
    while (f(x + alpha * step) - f(x) >= 0):
    #while (f(x + alpha * h) - f(x) > q * alpha * grad(f,x).dot(h)):
        alpha *= l
    return alpha

def gradient_descent(f,x0,h=0.001):
    """
    Gradient method, used to minimize function f numerically, 
    starting from the point x0 and computing gradiendt with o(h)
    Returns approximate minimum
    """
    print('The initial point is ({x}, {y})\n\n'.format(x=x0[0], y=x0[1]))
    x_new = x0
    k = 0
    x = [x_new]
    while True:
        x_old = x_new    
        step = - grad(f,x_old,h)
        alpha = alpha_split(f,x_old,step)
        x_new = x_old + alpha * step
        x.append(x_new)
        #print('{iter:>3}. alpha = {al:<17.15f},   x_{iter:<3} = ({x:>18.15f}, 
        #{y:>18.15f})\n'.format(iter=k, x=x_new[0], y=x_new[1], al=alpha))
        if (stop(f,x_old,x_new)):
            break
        break
    return x_new, x

### Stochastic gradient descent

In [396]:
def stochastic_gradient_step_linear(X, y, w, train_ind, eta):
    grad = np.zeros((len(w)))
    grad = [
        X[train_ind][i] * (np.array(X[train_ind]) @ w.T - y[train_ind]) / len(y) 
        for i in range(len(grad))
    ]
    return  w - eta * np.array(grad)

def stochastic_gradient_descent(data, output, step, w_init, cost, hypotesis, plotting_freq,  eta=0.3, 
                                max_iter=1e8, min_weight_dist=1e-11,
                                seed=42, verbose=False):
    
    X = np.hstack ((np.array(data), np.ones((len(output),1))))
    y = np.array(output)
    weight_dist = np.inf
    w = w_init
    errors = []
    iter_num = 0
    show_graphs(data, chosen_cols, w_init, output, hypotesis, clear_output=True)
    np.random.seed(seed)
    while weight_dist > min_weight_dist and iter_num < max_iter:
        random_ind = np.random.randint(X.shape[0])
        iter_num = iter_num + 1
        if (iter_num % plotting_freq == 0):
            show_graphs(data, chosen_cols, w_next, output, hypotesis, clear_output=True)
            show_errors(errors, cost.name, clear_output = True)
        w_next = stochastic_gradient_step_linear(X,y,w,random_ind, eta)
        errors.append(cost(y, hypotesis(data, w_next)))
        weight_dist = np.linalg.norm(w_next - w)
        w = w_next
    pl.clf() 
    return w, errors

In [397]:
def stochastic_gradient_step(X, y, w, k, eta=0.01):
    n = len(w)
    grad = 2 * (X[k].dot(w) - y[k]) * X[k,:]
    w_next = w.reshape((n,1)) - eta * grad.reshape((n,1)) / X.shape[0]    
    return w_next

def stochastic_gradient_descent(X, y, w_init=None, eta=1e-2, max_iter=1e4,
                                min_weight_dist=1e-8, seed=42, verbose=False):    
    X = np.array(X)
    y = np.array(y)
    if(w_init == None):
        w_init = np.zeros(X.shape[1])
        
    w = w_init
    weight_dist = np.inf
    errors = []
    iter_num = 0
    #import random
    #k = np.array(range(X.shape[0]))
    #random.shuffle(k)
    np.random.seed(seed)
        
    while weight_dist > min_weight_dist and iter_num < max_iter:  
        print('\nit.',iter_num)
        print('X: ',X[:5])
        print('y: ',y[:5])
        print('w: ',w)
        random_ind = np.random.randint(X.shape[0])
        print('k: ',random_ind)
        w_next = stochastic_gradient_step(X, y, w, random_ind, eta)        
        print('w1 ',w_next)
        errors.append(mean_squared_error(y, np.dot(X, w_next)))
        weight_dist = np.linalg.norm(w_next - w)      
        print('d: ',weight_dist)
        w = w_next
        iter_num += 1
        #if(iter_num == 5):
            #break
    return w, errors

In [398]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels, minimize='st_grad_desc')[0]
print_result(w,test_labels,linear_regression_prediction(test_data, w))
#plot_for_one_feature(train_data, train_labels, w, 'train')
#plot_for_one_feature(test_data, test_labels, w, 'test')


it. 0
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [ 0.  0.]
k:  102
w1  [[ 0.00174286]
 [ 0.36042286]]
d:  0.50972085204

it. 1
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ 0.00174286]
 [ 0.36042286]]
k:  92
w1  [[-0.00152517]
 [ 0.00159375]]
d:  0.358843988313

it. 2
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[-0.00152517]
 [ 0.00159375]]
k:  14
w1  [[ 0.00071786]
 [ 0.28354228]]
d:  0.281957447558

it. 3
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ 0.00071786]
 [ 0.28354228]]
k:  106
w1  [[-0.00170113]
 [ 0.06486546]]
d:  0.218690198832

it. 4
X:  [[   1.    50. ]
 [   1.   184.9]
 [ 

X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  1.03399235e+13]
 [  1.63249691e+15]]
k:  105
w1  [[ -5.09268306e+13]
 [ -1.44622794e+16]]
d:  1.60948929217e+16

it. 57
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -5.09268306e+13]
 [ -1.44622794e+16]]
k:  3
w1  [[ -1.74780541e+12]
 [ -1.32918186e+16]]
d:  1.17149351743e+15

it. 58
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -1.74780541e+12]
 [ -1.32918186e+16]]
k:  53
w1  [[  3.23522234e+14]
 [  4.24269391e+16]]
d:  5.57197071096e+16

it. 59
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  3.23522234e+14]
 [  4.24269391e+16]]
k:  43
w1  [[ 

X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -1.24672979e+41]
 [ -3.45691922e+43]]
k:  27
w1  [[  1.19341875e+42]
 [  3.17229489e+44]]
d:  3.51801150678e+44

it. 127
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  1.19341875e+42]
 [  3.17229489e+44]]
k:  107
w1  [[ -7.86138773e+42]
 [ -1.49192085e+45]]
d:  1.80917299431e+45

it. 128
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -7.86138773e+42]
 [ -1.49192085e+45]]
k:  43
w1  [[  2.20420917e+43]
 [  2.70353732e+45]]
d:  4.19556473268e+45

it. 129
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  2.20420917e+43]
 [  2.70353732e+45]]
k:  83
w1 

X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  1.88048760e+59]
 [  2.58879841e+61]]
k:  98
w1  [[ -8.52675066e+59]
 [ -2.66971700e+62]]
d:  2.92861533779e+62

it. 203
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -8.52675066e+59]
 [ -2.66971700e+62]]
k:  47
w1  [[  8.27406502e+60]
 [  1.91705720e+63]]
d:  2.18404797203e+63

it. 204
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  8.27406502e+60]
 [  1.91705720e+63]]
k:  130
w1  [[ -5.01699751e+61]
 [ -1.05549010e+64]]
d:  1.24720951001e+64

it. 205
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -5.01699751e+61]
 [ -1.05549010e+64]]
k:  53
w1 

X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -4.76421416e+80]
 [ -1.25395878e+83]]
k:  23
w1  [[  2.20532706e+81]
 [  2.76061869e+83]]
d:  4.01466704139e+83

it. 269
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  2.20532706e+81]
 [  2.76061869e+83]]
k:  113
w1  [[ -7.15742909e+81]
 [ -1.94665644e+84]]
d:  2.22273803048e+84

it. 270
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -7.15742909e+81]
 [ -1.94665644e+84]]
k:  31
w1  [[  4.67103581e+82]
 [  8.48753393e+84]]
d:  1.04343294247e+85

it. 271
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  4.67103581e+82]
 [  8.48753393e+84]]
k:  85
w1 

...

w1  [[ -9.24913660e+225]
 [ -1.51754021e+229]]
d:  inf

it. 824
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -9.24913660e+225]
 [ -1.51754021e+229]]
k:  9
w1  [[  4.62507982e+226]
 [ -1.37546038e+229]]
d:  inf

it. 825
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  4.62507982e+226]
 [ -1.37546038e+229]]
k:  4
w1  [[  6.17453237e+227]
 [  1.52293945e+230]]
d:  inf

it. 826
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  6.17453237e+227]
 [  1.52293945e+230]]
k:  122
w1  [[ -4.09069351e+228]
 [ -8.66549010e+230]]
d:  inf

it. 827
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -4.09069351e+228

k:  85
w1  [[  2.37431247e+245]
 [ -8.82917502e+245]]
d:  inf

it. 916
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  2.37431247e+245]
 [ -8.82917502e+245]]
k:  62
w1  [[  2.37914466e+245]
 [ -8.80936305e+245]]
d:  inf

it. 917
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  2.37914466e+245]
 [ -8.80936305e+245]]
k:  103
w1  [[  2.69820712e+245]
 [  7.21686885e+246]]
d:  inf

it. 918
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  2.69820712e+245]
 [  7.21686885e+246]]
k:  48
w1  [[  8.86387575e+244]
 [ -2.46168005e+247]]
d:  inf

it. 919
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  8.8638

d:  inf

it. 1000
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -5.79605136e+271]
 [ -6.02646025e+273]]
k:  13
w1  [[  9.44311185e+271]
 [  2.09468586e+274]]
d:  inf

it. 1001
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  9.44311185e+271]
 [  2.09468586e+274]]
k:  8
w1  [[ -3.07462816e+272]
 [ -3.30274968e+274]]
d:  inf

it. 1002
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[ -3.07462816e+272]
 [ -3.30274968e+274]]
k:  39
w1  [[  3.59264722e+272]
 [  6.11811043e+274]]
d:  inf

it. 1003
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  3.59264722e+272]
 [  6.11811043e+274]]
k:  24
w1  [[  2.08

 [ -1.06890809e+292]]
d:  inf

it. 1084
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  1.54411790e+290]
 [ -1.06890809e+292]]
k:  11
w1  [[  2.76856059e+290]
 [ -8.69050603e+290]]
d:  inf

it. 1085
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  2.76856059e+290]
 [ -8.69050603e+290]]
k:  61
w1  [[  2.78442874e+290]
 [ -8.48263323e+290]]
d:  inf

it. 1086
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  2.78442874e+290]
 [ -8.48263323e+290]]
k:  79
w1  [[  3.05038565e+290]
 [  4.99746951e+291]]
d:  inf

it. 1087
X:  [[   1.    50. ]
 [   1.   184.9]
 [   1.   112.9]
 [   1.    23.8]
 [   1.   290.7]]
y:  [[  8.4]
 [ 15.5]
 [ 11.9]
 [  9.2]
 [ 12.8]]
w:  [[  3.05038565e+290]
 [  4.99746951e+29

/Users/Yaroslava/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Yaroslava/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in subtract
  after removing the cwd from sys.path.
